In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame as DF
import matplotlib.pyplot as plt

In [2]:
import yfinance as yf
data = yf.download("AAPL", start="2018-01-01", end="2023-10-10",proxy="http://127.0.0.1:7890")
data

ModuleNotFoundError: No module named 'yfinance'

### 1.1  Indicatos

In [ ]:

def SMA(df, n):
    '''
    Simple Moving Average 简单移动平均线，典型长度为10、20、50
    '''
    df['SMA_' + str(n)] = pd.Series.rolling(df['Close'], n).mean()
SMA(data, 10) 

In [ ]:
def EMA(df, n):
    '''
    Exponential Moving Average 指数平滑移动平均线，span根据跨度指定衰减
    '''
    df['EMA_' + str(n)] = pd.Series.ewm(df['Close'],span=n, min_periods=n - 1).mean()
EMA(data,5)

In [ ]:
def MOM(df, n):
    '''
    Momentum  
    动量显示的是一段时间内价格变动的速度，以帮助投资者判断趋势的强弱。
    倾向于随动量的强弱而变动的股票被称为动量股。
    '''
    df['MOM_' + str(n)] = pd.Series(df['Close'].diff(n))
MOM(data, 5)

In [ ]:

def ROC(df, n):
    '''
    Rate of Change
    一般是用当前股票价格与过去一段时间的股票价格进行比较，计算出股票价格变动的百分比。
    ROC指标的计算公式如下：ROC =（当日收盘价- n 天前的收盘价）/n 天前的收盘价 × 100
    其中，n表示ROC指标的时间周期，可以根据需要设定，常用的时间周期包括12天、25天、50天等
    '''
    M = df['Close'].diff(n - 1)
    N = df['Close'].shift(n - 1)

    df['ROC_' + str(n)] = pd.Series(M / N)
ROC(data,12)

In [ ]:
def ATR(df, n):
    '''
    Average True Range
    均幅指标,是取一定时间周期内的股价波动幅度的移动平均值,显示市场变化率的指标,主要用于研判买卖时机
    一般选择14天
    '''
    TR_l = [0]
    for k in range(len(df.index)-1):
        TR = max(df['High'][k + 1], df['Close'][k]) - min(df['Low'][k + 1], df['Close'][k])
        TR_l.append(TR)

    TR_s = pd.Series(TR_l)

    df['ATR_' + str(n)] = pd.Series.ewm(TR_s, span=n,min_periods=n).mean().values
ATR(data, 14)

In [ ]:
def BBANDS(df, n, multiplier=2, middle=False):
    '''
    Bollinger Bands
    绘制价格的标准差通道来衡量价格波动的程度,并提供价格的上下限范围
    中轨是以移动平均线为基础计算得出的，通常采用简单移动平均线（SMA）。常用的周期长度为20天
    '''
    ma = pd.Series.rolling(df['Close'], n).mean()
    msd = pd.Series.rolling(df['Close'], n).std()

    b1 = 4 * msd / ma
    b2 = (df['Close'] - ma + multiplier * msd) / (4 * msd)

    df['BBANDSup_' + str(n)] = b1

    if middle:
        df['BBANDSmiddle_' + str(n)] = ma

    df['BBANDSdown_' + str(n)] = b2
    
BBANDS(data,20, multiplier=2, middle=False)

In [ ]:

def PPSR(df):
    '''
    Pivot Points, Supports and Resistances
    轴心点，在pp上下各有3条价格先，统计价格突破相应线的改了
    '''
    pp = pd.Series((df['High'] + df['Low'] + df['Close']) / 3)

    s1 = pd.Series(2 * pp - df['High'])
    s2 = pd.Series(pp - df['High'] + df['Low'])
    s3 = pd.Series(df['Low'] - 2 * (df['High'] - pp))

    r1 = pd.Series(2 * pp - df['Low'])
    r2 = pd.Series(pp + df['High'] - df['Low'])
    r3 = pd.Series(df['High'] + 2 * (pp - df['Low']))

    df['PP'] = pp
    df['S1'] = s1
    df['S2'] = s2
    df['S3'] = s3
    df['R1'] = r1
    df['R2'] = r2
    df['R3'] = r3
PPSR(data)

In [ ]:
def STOK(df):
    '''
    Stochastic oscillator %K
    随机指标(stochastic oscillator)又称KD指标，由一条%K线以及一条%D线构成。
    “%K”线代表最新价格和近期最低价的差，与近期最高最低差价的比
    '''
    df['STOK'] = (df['Close'] - df['Low']) / (df['High'] - df['Low'])
STOK(data)

In [ ]:

def STO(df, n):
    '''
    Stochastic oscillator %D
    “近期”这个时间范围是%K的三倍
    '''
    SOk = pd.Series((df['Close'] - df['Low']) / (df['High'] - df['Low']))

    df['STO_' + str(n)] = pd.Series.ewm(SOk, span=n, min_periods=n - 1).mean()

STO(data,10)

In [ ]:

def MACD(df, n_fast=12, n_slow=26):
    '''
    MACD, MACD Signal and MACD difference
    '''
    emaFast = pd.Series.ewm(df['Close'], span=n_fast,min_periods=n_slow - 1).mean()
    emaSlow = pd.Series.ewm(df['Close'], span=n_slow,min_periods=n_slow - 1).mean()
    macd = pd.Series(emaFast - emaSlow)
    macdSign = pd.Series.ewm(macd, span=9, min_periods=8).mean()
    macdDiff = pd.Series(macd - macdSign)

    df['MACD_' + str(n_fast) + '_' + str(n_slow)] = macd
    df['MACDsignal_' + str(n_fast) + '_' + str(n_slow)] = macdSign
    df['MACDdiff_' + str(n_fast) + '_' + str(n_slow)] = macdDiff

MACD(data,10)

In [ ]:
def MFI(df, n):
    '''
    Money Flow Index and Ratio
    衡量资金流动的强度和方向
    '''
    pp = (df['High'] + df['Low'] + df['Close']) / 3
    PosMF = [0]
    for k in range(len(df.index) - 1):
        if pp[k + 1] > pp[k]:
            PosMF.append(pp[k + 1] * df['Volume'][k + 1])
        else:
            PosMF.append(0)

    PosMF = pd.Series(PosMF)
    TotMF = pp * df['Volume']

    # .values was used beacause in a nonsense way PosMF/TotMF was
    # generating a double size dataFrame and the first half had datas as index
    # ! We got an RuntimeWarning because division buy zero, but it still works
    mfr = pd.Series(PosMF.values / TotMF.values)

    df['MFI_' + str(n)] = pd.Series.rolling(mfr, n).mean().values

MFI(data,10)

In [ ]:
def OBV(df, n):
    '''
    On-balance Volume平均成交量（on balance volume）
    从基准日起对每天的交易量进行统计，将股价上升日的交易量加入累计值，将股价下跌日的交易量从累计值中减去，股价不变交易量不变
    '''
    obv = [0]
    for k in range(len(df.index) - 1):
        if df['Close'][k + 1] - df['Close'][k] > 0:
            obv.append(df['Volume'][k + 1])
        if df['Close'][k + 1] - df['Close'][k] == 0:
            obv.append(0)
        if df['Close'][k + 1] - df['Close'][k] < 0:
            obv.append(-df['Volume'][k + 1])

    obv = pd.Series(obv)
    df['OBV_' + str(n)] = pd.Series.rolling(obv, n).mean().values

OBV(data, 10)

In [ ]:
# delete NaN
data.dropna(inplace=True)
data

In [ ]:
data.to_csv('Apple_data.csv')

In [ ]:
data['Close'].plot(figsize=(16,4),label='APPL')